# Continuous PASTIS on HiCAT

Checking and testing various functions for this implementation.

In [ ]:
# Imports
import os
from astropy.io import fits
import hcipy
import matplotlib.pyplot as plt
import numpy as np

import hicat
from hicat.wfc_algorithms.wfsc_utils import actuator_grid, dm_actuators_to_surface

from pastis.config import CONFIG_PASTIS
from pastis.e2e_simulators.hicat_imaging import read_continuous_dm_maps_hicat

## Boston DM command combinations

I need to be able to poke each Boston DM actuator individually. There is the function `dm1.set_actuator()`, but that requires me to address each actuator by its coordinates (two numbers) rather by its index, which I would rather avoid due to compatibility with the rest of the PASTIS code. (It would probably be easy to do, but I don't want to add this conversion step if I can avoid it.)

What I could do instead is do make a (2D or 1D) DM command like the ones I read in from fits from the strokemin solutions, and aplpy that.

On top of this, I will need to aberrate each actuator on top of the strokemin solution for DM1 that I apply as part of my coronagraph, so by doing othe fits reading thing, I might be able to just add them, and apply the resulting DM command.

### Read and plot fits 2D commands

In [ ]:
# Read the strokemin solution for DM1 and Dm2
path_to_dh_solution = CONFIG_PASTIS.get('HiCAT', 'dm_maps_path')
print(path_to_dh_solution)

dm1_strokemin, dm2_strokemin = read_continuous_dm_maps_hicat(path_to_dh_solution)
print(type(dm1_strokemin))
print(dm1_strokemin.shape)

In [ ]:
# Plot both DM maps

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(dm1_strokemin)
plt.title('DM1 stroke min solution, read from 2D command')

plt.subplot(1, 2, 2)
plt.imshow(dm2_strokemin)
plt.title('DM2 stroke min solution, read from 2D command')

The imported function `read_continuous_dm_maps_hicat()` reads the 2D fits arrays containig the DM maps. Let's make a function that reads the equivalent 1D commands.

### Cast a DM command array (952) into a 2D DM command

I will be largely following what is implemented in the Jacobian generation from HiCAT to create my Dm commands that poke one individual actuator. Since the Jacobian script also implements a DM command input parameter around which to calculate the Jacobian, I can literally use the code from there.

In the following few cells I am just seeing how to poke individual actuators.

In [ ]:
dm1_actuators = np.zeros(952)
dm1_command = dm_actuators_to_surface(dm1_actuators).reshape(actuator_grid.shape)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(dm1_command)
plt.title('DM1 command')

In [ ]:
# Yes, this has the shape I need to apply to hicat_sim.dm1 - nice
print(dm1_strokemin.shape)
print(dm1_command.shape)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(dm1_strokemin)

plt.subplot(1, 2, 2)
plt.imshow(dm1_command)

In [ ]:
# Poke one actuator
actuator_num = 384
dm1_actuators[actuator_num] += 50 * 1e-9    # meters

dm1_command_one_poke = dm_actuators_to_surface(dm1_actuators).reshape(actuator_grid.shape)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(dm1_command_one_poke)
plt.title(f'DM1 command with actuator {actuator_num} poked')

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(dm1_strokemin)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(dm1_command_one_poke)
plt.colorbar()

### Combine two commands - one DM array and one full 2D command

I think this means that I can create a 952 actuator array in which I poke the actuator I want, convert it to a 2D DM command and then add it to the stroke min solution. This co-added result is what I then need to load on my DM.

In [ ]:
final_command_to_load = dm1_strokemin + dm1_command_one_poke

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(dm1_strokemin)
plt.title('Without extra poke')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(final_command_to_load)
plt.title(f'Wiith actuator {actuator_num} poked')
plt.colorbar()